In [3]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 71.2 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')  

[nltk_data] Downloading package punkt to /home/garuda/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/garuda/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/garuda/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/garuda/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [9]:
import re
import string
import PyPDF2
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk


def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + " "
    return text

def preprocess(text):
    text = text.lower()
    text = re.sub(f'[{re.escape(string.punctuation)}0-9\r\t\n]', ' ', text)
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if w not in stop_words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(w) for w in words]
    return ' '.join(words)

doc1_text = extract_text_from_pdf("d1/1.pdf")
doc2_text = extract_text_from_pdf("d1/2.pdf")
doc1_processed = preprocess(doc1_text)
doc2_processed = preprocess(doc2_text)

documents = [doc1_processed, doc2_processed]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
print(f'Cosine Similarity: {cosine_sim[0][0]}')

euclidean_dist = euclidean_distances(tfidf_matrix[0:1], tfidf_matrix[1:2])
print(f'Euclidean Distance: {euclidean_dist[0][0]}')

def jaccard_similarity(doc1, doc2):
    words_doc1 = set(doc1.split())
    words_doc2 = set(doc2.split())
    intersection = words_doc1.intersection(words_doc2)
    union = words_doc1.union(words_doc2)
    return float(len(intersection)) / len(union)

jaccard_sim = jaccard_similarity(doc1_processed, doc2_processed)
print(f'Jaccard Similarity: {jaccard_sim}')


Cosine Similarity: 0.44869597385995824
Euclidean Distance: 1.0500514522060482
Jaccard Similarity: 0.17217310376919498
